In [9]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, SQLiteSession, function_tool
from agents.mcp import MCPServerStdio
import os

In [10]:
load_dotenv(override=True)

True

In [11]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

In [12]:
# Configure Playwright to run in non-headless mode
playwright_env = {"PLAYWRIGHT_HEADLESS": "false"}
playwright_params = {"command": "npx","args": [ "@playwright/mcp@latest"], "env": playwright_env}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.list_tools()

playwright_tools


[Tool(name='browser_close', title=None, description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Close browser', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_resize', title=None, description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_console_messages', title=None, desc

In [13]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()

file_tools

[Tool(name='read_file', title=None, description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'type': 'number', 'description': 'If provided, returns only the last N lines of the file'}, 'head': {'type': 'number', 'description': 'If provided, returns only the first N lines of the file'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=None, meta=None),
 Tool(name='read_text_file', title=None, description="Read the complete contents of a file from the file system as text. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a file. Operates 

In [14]:
@function_tool
def get_credentials(site_name: str):
    """ Takes site name as input and gives username and passowrd credentials of the site"""
    creds = dotenv_values(".env")
    username_key = f"{site_name.upper()}_USERNAME"
    password_key = f"{site_name.upper()}_PASSWORD"

    if username_key in creds and password_key in creds:
        return creds[username_key], creds[password_key]
    else:
        raise ValueError(f"Credentials for {site_name} not found in {env_file}")

In [15]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
If user asks to login into a particular website,
You may read credentials from the .env file using the provided tool `get_credentials`.
Always use the right credentials when logging into websites.
"""


async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_server_browser:
        
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model="gpt-4.1-mini",
            mcp_servers=[mcp_server_files, mcp_server_browser],
            tools=[get_credentials],
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Login into VTOP", max_turns=20)
            print(result.final_output)

I couldn't retrieve the credentials automatically. Could you please provide me with your VTOP login credentials (username and password) so I can log in for you?
